<a href="https://colab.research.google.com/github/gfeyzakorkmaz/gfeyzakorkmaz/blob/main/pythonday42.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
path = '/content/drive/MyDrive/Dataset/complex_marketing_data_enriched.xlsx'

In [4]:
df = pd.read_excel(path)

In [5]:
import pymc as pm
import numpy as np

conversions = df['Converted'].values
trials = np.ones_like(conversions)

with pm.Model() as model:
    prior = pm.Beta("prior", alpha=1, beta=1)
    likelihood = pm.Bernoulli("likelihood", p=prior, observed=conversions)
    trace = pm.sample(1000, return_inferencedata=False)


posterior = trace["prior"]
print("Posterior Mean Conversion Rate:", np.mean(posterior))

Output()

Output()

Posterior Mean Conversion Rate: 0.5928027215466594


This result represents the average 59.28 % of the customers converted(e.g., made a purchase...)

In [7]:
spend = df['Total Spent'].values
conversion = df['Converted'].values

with pm.Model() as model:
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=10)
    sigma = pm.HalfNormal("sigma", sigma=10)



    mu = alpha + beta * spend
    likelihood = pm.Normal("likelihood", mu=mu, sigma=sigma, observed=conversion)

    trace = pm.sample(1000, return_inferencedata=False)


print("Posterior Beta (Effect of Spend on Conversion):", np.mean(trace["beta"]))

Output()

Output()

Posterior Beta (Effect of Spend on Conversion): 0.00025425746078070176


Very small value

In [10]:
regions = pd.Categorical(df['Region']).codes
conversions = df['Converted'].values

with pm.Model() as model:
   region_effect = pm.Normal("region_effect", mu=0, sigma=1, shape=len(set(regions)))
   likelihood = pm.Bernoulli("likelihood", p=pm.math.sigmoid(region_effect[regions]), observed=conversions)
   trace = pm.sample(1000, return_inferencedata=False)


print("Regional Effects:", trace["region_effect"].mean(axis=0))

Output()

Output()

Regional Effects: [0.16021638 0.5240159  0.43109361 0.00978514]


In [13]:
campaigns = pd.Categorical(df['Campaign']).codes
conversions = df['Converted'].values

with pm.Model() as model:
    campaign_effect = pm.Normal("campaign_effect", mu=0, sigma=1, shape=len(set(campaigns)))
    likelihood = pm.Bernoulli("likelihood", p=pm.math.sigmoid(campaign_effect[campaigns]), observed=conversions)
    trace = pm.sample(1000, return_inferencedata=False)


print("Campaign Effects:", trace["campaign_effect"].mean(axis=0))

Output()

Output()

Campaign Effects: [-1.09808105  0.02385691  0.84046457  0.7114788 ]


In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
spending_scaled = scaler.fit_transform(df[['Total Spent']])


with pm.Model() as model:
    cluster_prob = pm.Dirichlet("cluster_prob", a=np.ones(3))
    cluster_assignment = pm.Categorical("cluster_assignment",p=cluster_prob, shape=len(spending_scaled))
    mu = pm.Normal("mu", mu=0, sigma=10, shape=3)
    likelihood = pm.Normal("likelihood", mu=mu[cluster_assignment], sigma=1, observed=spending_scaled.flatten())
    trace = pm.sample(1000, target_accept=0.95, return_inferencedata=False)


print("Cluster Means:", trace["mu"].mean(axis=0))

Output()

Output()

ERROR:pymc.stats.convergence:There were 100 divergences after tuning. Increase `target_accept` or reparameterize.
ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Cluster Means: [ 0.45732268 -0.68777792 -0.23011788]


In [19]:
import pymc as pm

with pm.Model() as model:
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=10)
    sigma = pm.HalfNormal("sigma", sigma=10)


    mu = alpha + beta * df['Customer Tenure (Years)']
    likelihood = pm.Normal("likelihood", mu=mu, sigma=sigma, observed=df['Total Spent'])

    trace = pm.sample(1000, return_inferencedata=False)


print("posterior Beta (Effect of Tenure on Spending):", np.mean(trace["beta"]))

Output()

Output()

posterior Beta (Effect of Tenure on Spending): 41.2847823493823


In [32]:
import pymc as pm
import numpy as np

time = np.arange(len(df))
spend = df['Total Spent'].values


with pm.Model() as model:
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=10)
    sigma = pm.HalfNormal("sigma", sigma=10)

    mu = alpha + beta * time
    likelihood = pm.Normal("likelihood", mu=mu, sigma=sigma, observed=spend)

    trace = pm.sample(1000, return_inferencedata=False)


print("Trend (Beta):", np.mean(trace["beta"]))

Output()

Output()

Trend (Beta): 31.450539706498716


In [34]:
import pandas as pd
import pymc as pm
import numpy as np

with pm.Model() as model:
     alpha = pm.Normal("alpha", mu=0, sigma=10)
     beta = pm.Normal("beta", mu=0, sigma=10)
     sigma = pm.HalfNormal("sigma", sigma=10)

     mu = alpha + pm.math.dot(df.drop(columns=['Converted']), beta)
     likelihood = pm.Bernoulli("likelihood", p=pm.math.sigmoid(mu), observed=df['Converted'])


     trace = pm.sample(1000, return_interencedata=False)

TypeError: Unsupported dtype for TensorType: object

In [35]:
with pm.Model() as model:
    region_campaign = pm.Normal("region_campaign", mu=0, sigma=1, shape=(len(set(df['Region'])), len(set(df['Campaign']))))
    likelihood = pm.Bernoulli("likelihood", p=pm.math.sigmoid(region_campaign), observed=df['Converted'])
    trace = pm.sample(1000, return_inferencedata=False)


print("Region-Campaign Effects:", trace["region_campaign"].mean(axis=0))

ValueError: Size length is incompatible with batched dimensions of parameter 0 Sigmoid.0:
len(size) = 1, len(batched dims Sigmoid.0) = 2. Size must be None or have length >= 2

In [36]:
import pandas as pd
import pymc as pm
import numpy as np

with pm.Model() as model:
    region_idx = pd.Categorical(df['Region']).codes
    campaign_idx = pd.Categorical(df['Campaign']).codes


    region_campaign = pm.Normal("region_campaign", mu=0, sigma=1, shape=(len(set(df['Region'])), len(set(df['Campaign']))))


    mu = region_campaign[region_idx, campaign_idx]


    likelihood = pm.Bernoulli("likelihood", p=pm.math.sigmoid(mu), observed=df['Converted'])

    trace = pm.sample(1000, return_inferencedata=False)

print("Region-Campaign Effects:", trace["region_campaign"].mean(axis=0))

Output()

Output()

Region-Campaign Effects: [[-0.69206655 -0.0176196   0.43597287  0.42803181]
 [-0.45198726  0.01023304  0.92891811  0.01205894]
 [-0.00358927  0.01640852  0.02000731  0.41066945]
 [-0.42988875  0.00328344  0.00364232  0.27390067]]


In [38]:
import pandas as pd
import numpy as np
import pymc as pm

with pm.Model() as model:
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=10)

    mu = pm.math.exp(alpha + beta * df['Total Spent'])
    likelihood = pm.Poisson("likelihood", mu=mu, observed=df['Stock'])

    trace = pm.sample(1000, return_inferencedata=False)



print("Effect of Spend on Stock:", np.mean(trace["beta"]))

Output()

Output()

Effect of Spend on Stock: 0.001756504565758331
